# Filters
The Google Charts API permits the creation of a dashboard with controls, which allows the user to filter the data displayed in the chart.  JoogleChart supports the addition of one or more filters to a chart.

A filter can "bind" the data or another filter.  When a filter binds the data, the filter affects only the data displayed.  When a filter binds another filter, it affects both the data displayed and the filtering options in the other filter.  By default, a filter in JoogleChart binds the data.  To bind another filter, the binding must be explicitly set.

### Filter types
Google Charts currently offers the following filters.  All types are supported in JoogleCharts.

- CategoryFilter
- NumberRangeFilter
- DateRangeFilter
- ChartRangeFilter
- StringFilter

### Creating a filter
1.  Create a filter object
2.  Add filter options, as found in the Google Charts documenation.
3.  Add the filter to the chart
4.  Bind another filter, if applicable.

### Setting filter state
We can initialize the filter control with state properties if we want.  See the Google Charts documentation for each control's state properties.

In [9]:
import random
import pandas as pd
from jooglechart import JoogleChart, Filter, SeriesFilter

Following are some examples of Filter usage.

## CategoryFilter

In [10]:
list1 = []
list1.append(['Salesperson', 'State', 'Sales'])
list1.append(['Bobby', 'TX', 34])
list1.append(['Bobby', 'LA', 23])
list1.append(['Bobby', 'CO', 84])
list1.append(['Marsha', 'TX', 62])
list1.append(['Marsha', 'IL', 46])
list1.append(['Marsha', 'OK', 68])
list1.append(['Cindy', 'KY', 24])
list1.append(['Cindy', 'LA', 86])
list1.append(['Cindy', 'OK', 45])

table = JoogleChart(list1, chart_type="Table")

# create the filter object
filter1 = Filter(type="CategoryFilter")

# set the filter options
filter1.add_options(filterColumnIndex=0, ui_allowMultiple=True, ui_allowNone=True)

table.add_filter(filter1)

table.show()


## CategoryFilter: binding another filter

In [11]:
# binding a second filter to the first will cause the first filter to 
# control the selectable options in the second.

filter2 = Filter(type="CategoryFilter")
filter2.add_options(filterColumnIndex=1, ui_allowMultiple=True, ui_allowNone=True)

table.add_filter(filter2)

filter1.bind_filter(filter2)

table.show()

## CategoryFilter:  filtering a non-table chart
To filter on a non-display column, you'll need to set the viewable columns on the chart with the **set_view_cols()** method.  In essence, you're hiding the filtering column.  Doing this keeps the charted data in accord with the data model required for the desired chart type.  Omitting this step will produce an error message from Google Charts.

In [12]:
# Create a dataframe with two categories and a couple of data series
state = ['CA', 'CA', 'CA', 'TX', 'TX', 'TX']
city = ['Modesto', 'Sacramento', 'Redlands', 'Flugerville', 'Laredo', 'El Paso']
coconuts = [35, 65, 44, 76, 24, 46]
mangos = [23, 64, 23, 24, 75, 24]
df = pd.DataFrame({'state': state, 'city': city, 'coconuts': coconuts, 'mangos': mangos},
                  columns=['state', 'city', 'coconuts', 'mangos'])

chart = JoogleChart(df, chart_type="ColumnChart")

filter1 = Filter(type="CategoryFilter")
filter1.add_options(filterColumnIndex=0, ui_allowMultiple=True, ui_allowNone=True)

chart.add_filter(filter1)

# REQURIED:  set the viewable columns
chart.set_view_cols([1, 2, 3])

chart.show()

## NumberRangeFilter

In [13]:
cities = ['New York', 'Chicago', 'Dallas', 'Boston']
pies = [12, 32, 5, 20]
pies_df = pd.DataFrame({'cities': cities, 'pies': pies}, columns=['cities', 'pies'])

pie = JoogleChart(pies_df, chart_type="PieChart")

filter1 = Filter(type="NumberRangeFilter")
filter1.add_options(filterColumnIndex=1)

pie.add_filter(filter1)

pie.show()


## NumberRangeFilter:  adding state

In [14]:
# We will add state values to the filter we created above.  Notice that we can modify the filter object
# at any time, even after it has been added to the chart object.
filter1.add_state(lowValue=10, highValue=25)

pie.show()

## DateRangeFilter
To add lowValue and highValue to a DateRangeFilter, pass a python date or datetime object.  Pandas Timestamp objects will also work.

In [15]:
# Create a dataframe with a series of dates
periods = 20
dates = pd.date_range('2016-01-01', periods=periods, freq='D')

random.seed(1)
widgets = [random.randint(10, 20) for i in range(periods)]

data = {}
data['dates'] = dates
data['widgets'] = widgets
columns = ['dates', 'widgets']

df = pd.DataFrame(data, columns=columns)

date_chart = JoogleChart(df, chart_type = "LineChart")

# Create DateRangeFilter and add state values as date objects
filter1 = Filter("DateRangeFilter")
filter1.add_options(filterColumnIndex=0)

low_value = pd.to_datetime('2016-01-07')
filter1.add_state(lowValue=low_value)

high_value = pd.to_datetime('2016-01-15')
filter1.add_state(highValue=high_value)

date_chart.add_filter(filter1)

date_chart.show()

## Series Filter
JoogleCharts has a custom filter that allows the user to filter on columns of the data, rather than rows.  The data model used in the chart must have a column of categories, followed by columns with data series, as with a line chart or column chart.  If a SeriesFilter is applied to a line chart, for example, lines will appear or disappear as the series of data are selected or deselected.

The SeriesFilter will be most effective with a chart like line chart or ***stacked*** column chart, where series can be toggled on and off without leaving gaps along the axis where the data is now "missing."

The SeriesFilter under the hood is a re-purporsed CategoryFilter, so any options CategoryFilter takes can be applied to SeriesFilter.  However, not all will be relevant; filterColumnLabel and filterColumnIndex cannot be used.

By default, all available series will be selected on rendering.  The filter can be initialized with selections by passing SelectedValues as added filter state.  The default label for the filter is "Columns," which can be overriden with the ui.label filter option.

SeriesFilter can be combined with other types of filters.  In that case, the SeriesFilter will be rendered on top.

SeriesFilter will work even when the view_cols have been set (i.e., where some columns have been hidden), and when columns have been set as roles, including tooltips.

In [28]:
bradys = ['Bobby', 'Cindy', 'Jan']
snickers = [34, 64, 24]
mounds = [20, 32, 41]
milky_way = [35, 32, 50]
payday = [23, 64, 27]

d = {}
d['Kids'] = bradys
d['Snickers'] = snickers
d['Mounds'] = mounds
d['Milky Way'] = milky_way
d['Payday'] = payday
sf_df = pd.DataFrame(d, columns=['Kids', 'Snickers', 'Mounds', 'Milky Way', 'Payday'])

chart = JoogleChart(sf_df, chart_type="ColumnChart")
chart.add_chart_options(isStacked=True)
sf = SeriesFilter()
sf.add_options(ui_label="Candy Bars")
chart.add_filter(sf)
chart.show()